In [3]:
USER="--user"
!pip install {USER} google-cloud-aiplatform

  Using cached google_cloud_aiplatform-1.20.0-py2.py3-none-any.whl (2.3 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
#from google.cloud.aiplatform_v1 import (FeaturestoreOnlineServingServiceClient,FeaturestoreServiceClient)
from google.cloud.aiplatform_v1.types import FeatureSelector, IdMatcher
from google.cloud.aiplatform_v1.types import entity_type as entity_type_pb2
from google.cloud.aiplatform_v1.types import feature as feature_pb2
from google.cloud.aiplatform_v1.types import featurestore as featurestore_pb2
from google.cloud.aiplatform_v1.types import featurestore_online_service as featurestore_online_service_pb2
from google.cloud.aiplatform_v1.types import \
    featurestore_service as featurestore_service_pb2
from google.cloud.aiplatform_v1.types import io as io_pb2
from google.protobuf.duration_pb2 import Duration
import datetime

ModuleNotFoundError: No module named 'google.cloud.aiplatform_v1'

In [17]:
Project_id="vertex-ai-gcp-1"
featurestore_name="employee_fs_s6"
Entity_name="emp_entity_s6"
location = "us-central1"
endpoint = "us-central1-aiplatform.googleapis.com"

In [7]:
import google.cloud.aiplatform_v1
client_admin = google.cloud.aiplatform_v1.FeaturestoreServiceClient(client_options={"api_endpoint": endpoint})
client_data = google.cloud.aiplatform_v1.FeaturestoreOnlineServingServiceClient(client_options={"api_endpoint": endpoint}) 
fs_resource_path = client_admin.common_location_path(Project_id, location)
print(fs_resource_path)
print(client_admin)
print(client_data)

projects/vertex-ai-gcp-1/locations/us-central1


In [18]:
#Creation of feature store
from google.cloud.aiplatform_v1.types import featurestore_service as fs_s
from google.cloud.aiplatform_v1.types import featurestore as fs
from google.cloud.aiplatform_v1.types import feature
create_fs = client_admin.create_featurestore(
    fs_s.CreateFeaturestoreRequest(
        parent=fs_resource_path,
        featurestore_id=featurestore_name,
        featurestore=fs.Featurestore(
            online_serving_config=fs.Featurestore.OnlineServingConfig(
                fixed_node_count=1
            ),
        ),
    )
)
print(create_fs.result())
client_admin.get_featurestore(name=client_admin.featurestore_path(Project_id, location, featurestore_name))

name: "projects/953811166431/locations/us-central1/featurestores/employee_fs_s6"



name: "projects/953811166431/locations/us-central1/featurestores/employee_fs_s6"
create_time {
  seconds: 1671996104
  nanos: 507089000
}
update_time {
  seconds: 1671996104
  nanos: 951866000
}
etag: "AMEw9yPQmaXnMhci9bO0lbkvKrchDfvuV-vyMRwhi17zefPkRTB6lNYl_KL2f9CBV9mh"
online_serving_config {
  fixed_node_count: 1
}
state: STABLE

In [20]:
from google.cloud.aiplatform_v1.types import entity_type
entity_creation = client_admin.create_entity_type(
    fs_s.CreateEntityTypeRequest(
        parent=client_admin.featurestore_path(Project_id, location, featurestore_name),
        entity_type_id=Entity_name,
        entity_type=entity_type.EntityType(
            description="employee entity",
        ),
    )
)

# Similarly, wait for EntityType creation operation.
print(entity_creation.result())

name: "projects/953811166431/locations/us-central1/featurestores/employee_fs_s6/entityTypes/emp_entity_s6"



In [22]:
client_admin.batch_create_features(
    parent=client_admin.entity_type_path(Project_id, location, featurestore_name, Entity_name),
    requests=[
        fs_s.CreateFeatureRequest(
            feature=feature.Feature(
                value_type=feature.Feature.ValueType.INT64,
                description="employee id",
            ),
            feature_id="employee_id",
        ),
        fs_s.CreateFeatureRequest(
            feature=feature.Feature(
                value_type=feature.Feature.ValueType.STRING,
                description="education",
            ),
            feature_id="education",
        ),
        fs_s.CreateFeatureRequest(
            feature=feature.Feature(
                value_type=feature.Feature.ValueType.STRING,
                description="gender",
            ),
            feature_id="gender",
        ),
        fs_s.CreateFeatureRequest(
            feature=feature.Feature(
                value_type=feature.Feature.ValueType.INT64,
                description="no_of_trainings",
            ),
            feature_id="no_of_trainings",
        ),
        fs_s.CreateFeatureRequest(
            feature=feature.Feature(
                value_type=feature.Feature.ValueType.INT64,
                description="age",
            ),
            feature_id="age",
        ),
    ],
).result()

features {
  name: "projects/953811166431/locations/us-central1/featurestores/employee_fs_s6/entityTypes/emp_entity_s6/features/employee_id"
}
features {
  name: "projects/953811166431/locations/us-central1/featurestores/employee_fs_s6/entityTypes/emp_entity_s6/features/education"
}
features {
  name: "projects/953811166431/locations/us-central1/featurestores/employee_fs_s6/entityTypes/emp_entity_s6/features/gender"
}
features {
  name: "projects/953811166431/locations/us-central1/featurestores/employee_fs_s6/entityTypes/emp_entity_s6/features/no_of_trainings"
}
features {
  name: "projects/953811166431/locations/us-central1/featurestores/employee_fs_s6/entityTypes/emp_entity_s6/features/age"
}

In [52]:
from google.cloud.aiplatform_v1.types.featurestore_service import ImportFeatureValuesRequest

In [25]:
from google.cloud.aiplatform_v1.types import io
from google.protobuf.timestamp_pb2 import Timestamp
from google.cloud.aiplatform_v1.types.featurestore_service import ImportFeatureValuesRequest
import datetime
t = datetime.datetime.now().timestamp()
seconds = int(t)
#nanos = int(t % 1 * 1e9)
timestamp_input = Timestamp(seconds=seconds)
ingest_data_csv = fs_s.ImportFeatureValuesRequest(
    entity_type=client_admin.entity_type_path(
        Project_id, location, featurestore_name, Entity_name
    ),
    csv_source=io.CsvSource(
        # Source
        gcs_source=io.GcsSource(
            uris=[
                "gs://feature_store_input/employee_promotion_data_fs.csv"
            ]
        )
    ),
    entity_id_field="employee_id",
    feature_specs=[
        # Features
        ImportFeatureValuesRequest.FeatureSpec(id="employee_id"),
        ImportFeatureValuesRequest.FeatureSpec(id="education"),
        ImportFeatureValuesRequest.FeatureSpec(id="gender"),
        ImportFeatureValuesRequest.FeatureSpec(id="no_of_trainings"),
        ImportFeatureValuesRequest.FeatureSpec(id="age"),
    ],
    #feature_time_field=column_name containing the timestamp information in csv or in big query.
    feature_time=timestamp_input, #time stamp needs to be in google.protobuf.Timestamp
    worker_count=1,
)

In [26]:
ingest_data = client_admin.import_feature_values(ingest_data_csv)
ingest_data.result()

imported_entity_count: 49
imported_feature_value_count: 241

In [58]:
#The output is an iterator
from google.cloud.aiplatform_v1.types import FeatureSelector, IdMatcher
from google.cloud.aiplatform_v1.types import featurestore_online_service 
resp_data = client_data.streaming_read_feature_values(
    featurestore_online_service.StreamingReadFeatureValuesRequest(
        entity_type=client_admin.entity_type_path(
            Project_id, location, featurestore_name, Entity_name
        ),
        entity_ids=["65438"],
        feature_selector=FeatureSelector(id_matcher=IdMatcher(ids=["employee_id","education","gender","no_of_trainings","age"])),
    )
)
print(resp_data)
type(resp_data)

google.api_core.grpc_helpers._StreamingResponseIterator

In [31]:
for resp in resp_data:
    print(resp)

header {
  entity_type: "projects/953811166431/locations/us-central1/featurestores/employee_fs_s6/entityTypes/emp_entity_s6"
  feature_descriptors {
    id: "employee_id"
  }
  feature_descriptors {
    id: "education"
  }
  feature_descriptors {
    id: "gender"
  }
  feature_descriptors {
    id: "no_of_trainings"
  }
  feature_descriptors {
    id: "age"
  }
}

entity_view {
  entity_id: "65438"
  data {
    value {
      int64_value: 65438
      metadata {
        generate_time {
          seconds: 1671996186
        }
      }
    }
  }
  data {
    value {
      string_value: "Master\'s & above"
      metadata {
        generate_time {
          seconds: 1671996186
        }
      }
    }
  }
  data {
    value {
      string_value: "f"
      metadata {
        generate_time {
          seconds: 1671996186
        }
      }
    }
  }
  data {
    value {
      int64_value: 1
      metadata {
        generate_time {
          seconds: 1671996186
        }
      }
    }
  }
  data {

In [59]:
names_col=[]
for resp in resp_data:
    if resp.header.feature_descriptors != "":
        for head in resp.header.feature_descriptors:
            names_col.append(head.id)
    try:
        values=[]
        for items in resp.entity_view.data:
            if items.value.string_value !="":values.append(items.value.string_value)
            elif items.value.int64_value !="":values.append(items.value.int64_value)
    except:pass
print("Feature Names",names_col)
print("Feature Values",values)

Feature Names ['employee_id', 'education', 'gender', 'no_of_trainings', 'age']
Feature Values [65438, "Master's & above", 'f', 1, 35]


In [16]:
featurestore_name
client_admin.delete_featurestore(
    request=fs_s.DeleteFeaturestoreRequest(
        name=client_admin.featurestore_path(Project_id, location, featurestore_name),
        force=True,
    )
).result()
#client.delete_dataset(DESTINATION_DATA_SET, delete_contents=True, not_found_ok=True)  # Make an API request.

print("Deleted featurestore '{}'.".format(featurestore_name))

Deleted featurestore 'employee_fs_s5'.
